In [ ]:
using PyPlot
#load package for plotting
# package not installed? run Pkg.add("PyPlot.jl") from console

In [ ]:
a = [1;2;3] #a column vector

In [ ]:
b = [1 2 3] #a row vector

In [ ]:
ab = [a;b'] #concatenation

In [ ]:
A = [a b'] #create a matrix

In [ ]:
A[1,2],A[2,2] #access to an array (different parenthesis respect to matlab)

In [ ]:
println("   $(A[:,1])    $(A[1:2:end,1])    $(A[:])")

In [ ]:
A = speye(10)    # sparse diagonal matrix
typeof(A)        # you can have composite types
#A.nzval         # with their methods (e.g. non zero values)

In [ ]:
Nt,Nt2 = 120,100 #touples, similar to Phython 

In [ ]:
#let's build a function!
function create_toeplitz(Nt,Nt2)
    h = zeros(Nt)                 #initialize zero vector (in Matlab would be zeros(Nt,1))
    h[3],h[10],h[14] = 1.,0.7,0.5 #we build IR here and assign some values to IR
    H = zeros(Nt,Nt2)
    
    for i = 1:Nt2
        H[i:end,i] = h[1:end-i+1]   
    end
    return H,h                     #return multiple stuff
end
# no need to create external file for a function
# you can also create functions compactly e.g. f(x,y) = x^2+y^2
# you can also do this: g(x,y,f) = f(x,y)+x*y

In [ ]:
H,h = create_toeplitz(Nt,Nt2)
x = randn(Nt2)              #input random sequence
y = H*x                     #convolution

#plotting stuff,
#very very similar to matlab again
figure()
subplot(1,3,1)
plot(x)
title("Input signal x")
subplot(1,3,2)
plot(h)
title("Inpulse response h")
subplot(1,3,3)
plot(y)
title("Output y")

In [ ]:
x_tild = (H'*H+1e-8*speye(Nt2))\(H'*y)   #solve a regularized least squares  

#                      x_tild = argmin_x( 1/2|| H*x  -  y  ||_2^2 + 1/2|| x ||_2^2 )
# note the \ exactly like Matlab

ϵ = norm(x-x_tild)^2/norm(x) #can use greek letters (latex style) e.g. \epsilon+tab
println("error ϵ: $ϵ")


In [ ]:
#same function with other inputs
function create_toeplitz(Nt,Nt2,h)
    H = zeros(Nt,Nt2)
    
    for i = 1:Nt2
        H[i:end,i] = h[1:end-i+1]   
    end
    return H
end

h = randn(Nt).*exp(-0.05*(0:Nt-1)) #more "realisitc" IR

H = create_toeplitz(Nt,Nt2,h)
x = randn(Nt2)              #input random sequence
y0 = H*x
SNR = 30
y = y0+var(y0)*10^(-SNR/20)*randn(length(y0)) #add noise
x_tild = (H'*H+1e-8*speye(Nt2))\(H'*y)   #solve a regularized least squares

ϵ = norm(x-x_tild)^2/norm(x)^2 #can use greek letters (latex style) e.g. \epsilon+tab
println("error ϵ: $ϵ")

figure()
subplot(1,3,1)
plot(x, label = "x")
plot(x_tild, label = "x_tild")
legend()
title("Input signal x")
subplot(1,3,2)
plot(H[:,1])
title("Inpulse response h")
subplot(1,3,3)
plot(y,label = "y+noise")
plot(y0,label = "y")
legend()
title("Output y")


In [ ]:
#of course we have fft as well
yt,ht = fft([y;zeros(400)]),fft([h;zeros(400)]) #zero span
x_tild2 = real(ifft(yt./ht))[1:Nt2]

figure()
plot(x_tild,label="time dom.")
plot(x_tild2,label="freq. dom.")
plot(x,label="original")
legend()

ϵ2 = norm(x-x_tild2)^2/norm(x)^2 #can use greek letters (latex style) e.g. \epsilon+tab
println("error time domain (regularized) ϵ: $ϵ, frequency domain (unregularized) ϵ: $ϵ2")

In [ ]:
using DSP  #many packages are available such as DSP for filtering
Fs = 44e3
s = zeros(2000)
s[1] = 1

BP = digitalfilter(Bandpass(20,1000,fs = Fs),Butterworth(10))
s = filt(BP,s)
t = 0:1/Fs:((length(s)-1)/Fs)
f = linspace(0,Fs,length(s))

figure()
subplot(2,1,1)
plot(t,s)
subplot(2,1,2)
plot(f,10*log10(abs(fft(s))))
xlim([0;2e3])